# Rekurrenta neurala nätverk

I den föregående modulen har vi använt rika semantiska representationer av text och en enkel linjär klassificerare ovanpå embeddingarna. Vad denna arkitektur gör är att fånga den aggregerade betydelsen av ord i en mening, men den tar inte hänsyn till **ordningen** av orden, eftersom aggregeringsoperationen ovanpå embeddingarna tar bort denna information från den ursprungliga texten. Eftersom dessa modeller inte kan modellera ordningen av ord, kan de inte lösa mer komplexa eller tvetydiga uppgifter som textgenerering eller frågesvar.

För att fånga betydelsen av en textsekvens behöver vi använda en annan neural nätverksarkitektur, som kallas för ett **rekurrent neuralt nätverk**, eller RNN. I RNN skickar vi vår mening genom nätverket en symbol i taget, och nätverket producerar ett **tillstånd**, som vi sedan skickar tillbaka till nätverket tillsammans med nästa symbol.

Givet den inmatade sekvensen av tokens $X_0,\dots,X_n$, skapar RNN en sekvens av neurala nätverksblock och tränar denna sekvens från början till slut med hjälp av backpropagering. Varje nätverksblock tar ett par $(X_i,S_i)$ som indata och producerar $S_{i+1}$ som resultat. Slutligt tillstånd $S_n$ eller utdata $X_n$ skickas till en linjär klassificerare för att producera resultatet. Alla nätverksblock delar samma vikter och tränas från början till slut med en enda backpropageringspass.

Eftersom tillståndsvektorerna $S_0,\dots,S_n$ skickas genom nätverket, kan det lära sig de sekventiella beroendena mellan ord. Till exempel, när ordet *inte* dyker upp någonstans i sekvensen, kan det lära sig att negera vissa element inom tillståndsvektorn, vilket resulterar i negation.

> Eftersom vikterna för alla RNN-block i bilden är delade, kan samma bild representeras som ett enda block (till höger) med en rekurrent återkopplingsslinga, som skickar nätverkets utgångstillstånd tillbaka till ingången.

Låt oss se hur rekurrenta neurala nätverk kan hjälpa oss att klassificera vår nyhetsdatamängd.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Enkel RNN-klassificerare

När det gäller enkel RNN är varje återkommande enhet ett enkelt linjärt nätverk som tar en sammansatt inmatningsvektor och tillståndsvektor och producerar en ny tillståndsvektor. PyTorch representerar denna enhet med klassen `RNNCell`, och nätverk av sådana celler - som `RNN`-lager.

För att definiera en RNN-klassificerare kommer vi först att använda ett inbäddningslager för att minska dimensionen på inmatningsvokabulären, och sedan lägga till ett RNN-lager ovanpå:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Note:** Här använder vi ett otränat inbäddningslager för enkelhetens skull, men för ännu bättre resultat kan vi använda ett förtränat inbäddningslager med Word2Vec- eller GloVe-inbäddningar, som beskrivs i föregående avsnitt. För att få en bättre förståelse kan du anpassa denna kod för att fungera med förtränade inbäddningar.

I vårt fall kommer vi att använda en paddad dataladdare, så varje batch kommer att innehålla ett antal paddade sekvenser av samma längd. RNN-lagret kommer att ta sekvensen av inbäddningstensorer och producera två utgångar:
* $x$ är en sekvens av RNN-cellens utgångar vid varje steg
* $h$ är det slutliga dolda tillståndet för det sista elementet i sekvensen

Vi applicerar sedan en fullt ansluten linjär klassificerare för att få antalet klasser.

> **Note:** RNN:er är ganska svåra att träna, eftersom när RNN-cellerna rullas ut längs sekvenslängden blir det resulterande antalet lager som är involverade i backpropagering ganska stort. Därför behöver vi välja en liten inlärningshastighet och träna nätverket på en större dataset för att få bra resultat. Det kan ta ganska lång tid, så att använda GPU är att föredra.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Long Short Term Memory (LSTM)

Ett av de största problemen med klassiska RNN är det så kallade **vanishing gradients**-problemet. Eftersom RNN tränas från början till slut i en enda backpropagation-pass, har de svårt att föra vidare fel till de första lagren i nätverket, vilket gör att nätverket inte kan lära sig relationer mellan avlägsna tokens. Ett sätt att undvika detta problem är att införa **explicit tillståndshantering** genom att använda så kallade **grindar**. Det finns två mest kända arkitekturer av denna typ: **Long Short Term Memory** (LSTM) och **Gated Relay Unit** (GRU).

![Bild som visar ett exempel på en long short term memory-cell](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM-nätverket är organiserat på ett sätt som liknar RNN, men det finns två tillstånd som skickas från lager till lager: det faktiska tillståndet $c$ och den dolda vektorn $h$. Vid varje enhet kombineras den dolda vektorn $h_i$ med input $x_i$, och de styr vad som händer med tillståndet $c$ via **grindar**. Varje grind är ett neuralt nätverk med sigmoid-aktivering (output i intervallet $[0,1]$), vilket kan ses som en bitmask när den multipliceras med tillståndsvektorn. Följande grindar finns (från vänster till höger på bilden ovan):
* **Forget-grinden** tar den dolda vektorn och avgör vilka komponenter i vektorn $c$ vi behöver glömma och vilka vi ska föra vidare.
* **Input-grinden** tar viss information från input och den dolda vektorn och lägger till den i tillståndet.
* **Output-grinden** transformerar tillståndet via ett linjärt lager med $\tanh$-aktivering och väljer sedan vissa av dess komponenter med hjälp av den dolda vektorn $h_i$ för att producera ett nytt tillstånd $c_{i+1}$.

Komponenterna i tillståndet $c$ kan ses som flaggor som kan slås på och av. Till exempel, när vi stöter på namnet *Alice* i sekvensen, kanske vi vill anta att det hänvisar till en kvinnlig karaktär och höja flaggan i tillståndet som indikerar att vi har ett kvinnligt substantiv i meningen. När vi senare stöter på frasen *och Tom*, kommer vi att höja flaggan som indikerar att vi har ett plural-substantiv. Genom att manipulera tillståndet kan vi alltså hålla reda på grammatiska egenskaper hos meningens delar.

> **Note**: En utmärkt resurs för att förstå LSTM:s interna funktioner är den här fantastiska artikeln [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) av Christopher Olah.

Även om den interna strukturen hos en LSTM-cell kan verka komplex, döljer PyTorch denna implementation i klassen `LSTMCell` och tillhandahåller objektet `LSTM` för att representera hela LSTM-lagret. Implementeringen av en LSTM-klassificerare kommer därför att vara ganska lik den enkla RNN som vi har sett ovan:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Packade sekvenser

I vårt exempel var vi tvungna att fylla ut alla sekvenser i minibatchen med nollvektorer. Även om detta leder till viss minnesförlust, är det mer kritiskt med RNN att ytterligare RNN-celler skapas för de utfyllda inmatningsobjekten, som deltar i träningen men inte innehåller någon viktig inmatningsinformation. Det skulle vara mycket bättre att träna RNN endast på den faktiska sekvensstorleken.

För att göra detta introduceras ett speciellt format för lagring av utfyllda sekvenser i PyTorch. Anta att vi har en utfylld minibatch som ser ut så här:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Här representerar 0 utfyllda värden, och den faktiska längdvektorn för inmatningssekvenserna är `[5,3,1]`.

För att effektivt träna RNN med utfyllda sekvenser vill vi börja träna den första gruppen av RNN-celler med en stor minibatch (`[1,6,9]`), men sedan avsluta bearbetningen av den tredje sekvensen och fortsätta träningen med mindre minibatcher (`[2,7]`, `[3,8]`) och så vidare. Således representeras den packade sekvensen som en enda vektor - i vårt fall `[1,6,9,2,7,3,8,4,5]`, och längdvektorn (`[5,3,1]`), från vilken vi enkelt kan rekonstruera den ursprungliga utfyllda minibatchen.

För att skapa en packad sekvens kan vi använda funktionen `torch.nn.utils.rnn.pack_padded_sequence`. Alla rekurrenta lager, inklusive RNN, LSTM och GRU, stöder packade sekvenser som inmatning och producerar packad utmatning, som kan dekodas med hjälp av `torch.nn.utils.rnn.pad_packed_sequence`.

För att kunna skapa en packad sekvens måste vi skicka längdvektorn till nätverket, och därför behöver vi en annan funktion för att förbereda minibatcher:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Det faktiska nätverket skulle vara mycket likt `LSTMClassifier` ovan, men `forward`-passet kommer att ta emot både den vadderade minibatchen och vektorn av sekvenslängder. Efter att ha beräknat inbäddningen, beräknar vi den packade sekvensen, skickar den till LSTM-lagret och packar sedan upp resultatet igen.

> **Note**: Vi använder faktiskt inte det uppackade resultatet `x`, eftersom vi använder utdata från de dolda lagren i de följande beräkningarna. Därför kan vi ta bort uppackningen helt från denna kod. Anledningen till att vi placerar den här är för att du enkelt ska kunna modifiera denna kod, ifall du skulle behöva använda nätverkets utdata i vidare beräkningar.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Observera:** Du kanske har märkt parametern `use_pack_sequence` som vi skickar till träningsfunktionen. För närvarande kräver funktionen `pack_padded_sequence` att längdsekvenstensoren är på CPU-enheten, och därför måste träningsfunktionen undvika att flytta längdsekvensdata till GPU under träning. Du kan titta på implementeringen av funktionen `train_emb` i filen [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## Bidirektionella och flerskiktade RNN:er

I våra exempel har alla rekurrenta nätverk bearbetat data i en riktning, från början av en sekvens till slutet. Det känns naturligt, eftersom det liknar sättet vi läser och lyssnar på tal. Men i många praktiska fall har vi slumpmässig åtkomst till indata-sekvensen, och då kan det vara logiskt att köra rekurrent beräkning i båda riktningarna. Sådana nätverk kallas **bidirektionella** RNN:er, och de kan skapas genom att ange parametern `bidirectional=True` till konstruktören för RNN/LSTM/GRU.

När vi arbetar med ett bidirektionellt nätverk behöver vi två vektor för dold tillstånd, en för varje riktning. PyTorch kodar dessa vektorer som en enda vektor med dubbelt så stor storlek, vilket är ganska praktiskt eftersom du normalt skickar det resulterande dolda tillståndet till ett fullt anslutet linjärt lager. Du behöver bara ta hänsyn till denna storleksökning när du skapar lagret.

Ett rekurrent nätverk, oavsett om det är enkelriktat eller bidirektionellt, fångar vissa mönster inom en sekvens och kan lagra dem i en tillståndsvektor eller skicka dem vidare till utdata. Precis som med konvolutionella nätverk kan vi bygga ett annat rekurrent lager ovanpå det första för att fånga mönster på högre nivå, baserat på låg-nivå mönster som det första lagret har extraherat. Detta leder oss till begreppet **flerskiktad RNN**, som består av två eller fler rekurrenta nätverk, där utdata från det föregående lagret skickas som indata till nästa lager.

![Bild som visar en flerskiktad lång-korttidsminne-RNN](../../../../../translated_images/sv/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Bild från [detta fantastiska inlägg](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) av Fernando López*

PyTorch gör det enkelt att konstruera sådana nätverk, eftersom du bara behöver ange parametern `num_layers` till konstruktören för RNN/LSTM/GRU för att automatiskt bygga flera lager av rekurrens. Detta innebär också att storleken på den dolda tillståndsvektorn ökar proportionellt, och du måste ta hänsyn till detta när du hanterar utdata från de rekurrenta lagren.


## RNN:er för andra uppgifter

I denna enhet har vi sett att RNN:er kan användas för sekvensklassificering, men de kan faktiskt hantera många fler uppgifter, såsom textgenerering, maskinöversättning och mer. Vi kommer att titta på dessa uppgifter i nästa enhet.



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiska översättningar kan innehålla fel eller inexaktheter. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
